https://www.boots.com/

install Microsoft Edge WebDriver

In [1]:
# !pip install selenium

In [182]:
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import time

# url ของเว็บที่ต้องการดูข้อมูล
url = "https://www.boots.com/no7-future-renew-night-serum-25ml-10338927" 

# Set up EdgeOptions
edge_options = Options()
edge_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.54")
edge_options.add_argument("window-size=1920x1080")
edge_options.add_argument("disable-extensions")


# Create Edge WebDriver with headless options
driver = webdriver.Edge(options=edge_options)
driver.get(url)

In [143]:
# หา shadow host (element ที่มี shadow root)
shadow_host = driver.find_element(By.CSS_SELECTOR, '#estore_pdp_bottomsec_3 div[data-bv-show="reviews"]')

# เข้าถึง shadow root ผ่าน JavaScript
shadow_root = driver.execute_script('return arguments[0].shadowRoot', shadow_host)

In [144]:
print(f"shadow_root exists: {shadow_root is not None}")

shadow_root exists: True


In [148]:
button = (shadow_root.find_element(By.CLASS_NAME, 'iuGpzE')).text
button

'Show more'

In [149]:
# โหลดรีวิวทั้งหมด Load More
while True:
    try:
        # Scroll ลงล่างเพื่อให้ปุ่มแสดง
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

        # หา Load More
        load_more_button = shadow_root.find_element(By.CLASS_NAME, 'iuGpzE')

        if load_more_button.is_displayed() and load_more_button.is_enabled():
            print("Clicking Load More...")
            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(2)
        else:
            print("No more reviews to load.")
            break

    except NoSuchElementException:
        print("No Load More button found.")
        break

    except Exception as e:
        print("Unexpected error:", str(e))
        break

Clicking Load More...
Clicking Load More...
Clicking Load More...
Clicking Load More...
Clicking Load More...
Clicking Load More...
Clicking Load More...
No Load More button found.


In [ ]:
# relate = shadow_root.find_elements(By.CLASS_NAME, 'jolJQc')
# dates = [date.text for date in relate]
# dates

In [ ]:
# content = shadow_root.find_elements(By.CLASS_NAME,'gKsWtm')
# headers = [header.text for header in content]
# headers

In [ ]:
# content = shadow_root.find_elements(By.CLASS_NAME, 'ckDrNl')
# reviews = [review.text for review in content]
# reviews

In [163]:
review_blocks = shadow_root.find_elements(By.CLASS_NAME, 'bv-rnr__sc-16dr7i1-10')

for i, block in enumerate(review_blocks):
    print(f"---- Block {i+1} ----")
    print(block.text.strip())

---- Block 1 ----
4 out of 5 stars.
Night serum
Shoptillidrop
VERIFIED PURCHASE
10 hours ago
Saw improvement in skin overnight. Use tiny amount
Did you receive a free sample of this product?No
Yes, I would recommend this product.
Was this review helpful?
(0)
(0)
Report review
---- Block 2 ----
5 out of 5 stars.
Awesome night serum
Janet
VERIFIED PURCHASE
12 hours ago
Wow the difference is amazing and well Worth purchasing
Did you receive a free sample of this product?No
Yes, I would recommend this product.
Originally posted on No7 Future Renew Night Serum 50ml
Was this review helpful?
(0)
(0)
Report review
---- Block 3 ----
5 out of 5 stars.
Good so far
Traceg
3 days ago
I have only just started using this so hard to know. But it feels lovely on my skin so hopefully I will notice a difference soon.
Did you receive a free sample of this product?No
Yes, I would recommend this product.
Was this review helpful?
(0)
(0)
Report review
---- Block 4 ----
5 out of 5 stars.
Pleasant smooth feeli

In [ ]:
data = []

for r in review_blocks:
    lines = r.text.strip().split("\n")

    rating = lines[0].strip()
    title = lines[1].strip()
    date = lines[4].strip()
    review = lines[5].strip()

    data.append({
        "rating": rating,
        "title": title,
        "date": date,
        "review": review
    })

df = pd.DataFrame(data)
df.head()

In [175]:
len(df)

194

In [177]:
def convert_to_date(text):
    if not isinstance(text, str):
        return None
    
    text = text.strip().lower()
    today = datetime.today()

    # แบบ "x days/hours/months ago"
    match = re.match(r"(\d+)\s+(hour|day|week|month|year)s?\s+ago", text)
    if match:
        num = int(match.group(1))
        unit = match.group(2)

        if unit == "hour":
            return (today - relativedelta(hours=num)).strftime('%Y-%m-%d')
        elif unit == "day":
            return (today - relativedelta(days=num)).strftime('%Y-%m-%d')
        elif unit == "week":
            return (today - relativedelta(weeks=num)).strftime('%Y-%m-%d')
        elif unit == "month":
            return (today - relativedelta(months=num)).strftime('%Y-%m-%d')
        elif unit == "year":
            return (today - relativedelta(years=num)).strftime('%Y-%m-%d')

    # แบบ "12 May 2024"
    try:
        return datetime.strptime(text, "%d %B %Y").strftime('%Y-%m-%d')
    except:
        return None

In [178]:
df['date'] = df['date'].apply(convert_to_date)

In [179]:
df['source'] = 'boots.com'
df['product'] = 'No7 Future Renew Night Serum 25ml'

In [180]:
df.head()

,rating,title,date,review,source,product
0,4 out of 5 stars.,Night serum,2025-05-20,Saw improvement in skin overnight. Use tiny am...,boots.com,No7 Future Renew Night Serum 25ml
1,5 out of 5 stars.,Awesome night serum,2025-05-20,Wow the difference is amazing and well Worth p...,boots.com,No7 Future Renew Night Serum 25ml
2,5 out of 5 stars.,Good so far,None,Did you receive a free sample of this product?No,boots.com,No7 Future Renew Night Serum 25ml
3,5 out of 5 stars.,Pleasant smooth feeling cream,2025-05-17,This cream goes on really smoothly. I haven’t ...,boots.com,No7 Future Renew Night Serum 25ml
4,5 out of 5 stars.,No7 Future renew night Serum,2025-05-15,Works well ivernight. My skin looks more hydra...,boots.com,No7 Future Renew Night Serum 25ml


In [181]:
df.to_excel(f'{df['product'].iloc[0]}.xlsx', index=False)